In [1]:
import os
import pandas as pd
import numpy as np
import sys

from konlpy.tag import Kkma

sys.path.append('..')

from Code.config import *
from Code.preprocessingSentence import *

### 문제 List
* preprocessingentence 함수는 input sentence의 길이가 일정 수준 이상인 경우, null 값 return
    
    ```python 
    preprocessingSentence(df['content'].iloc[9], kkma)
    ```

In [ ]:
path = '../Data/Raw/탈모톡톡_19~23세탈모.csv'
df = pd.read_csv(path)

In [ ]:
kkma = Kkma()
result = df.iloc[:10].apply(lambda x : preprocessingSentence(x['title'] + x['content'], kkma), axis = 1)

### 오늘의 할 일
* multi-processing
    * multi-processing 함수 파라미터 제공
    * multi-processing 함수 jupyter notebook 실행
* save 함수 생성
* 감정분석 :  preprocessingSentence.py: sentimentAnalysis 선언
* LDA 
    * 주요기능
        * TF-IDF는 적정 크기의 키워드 수 선택이 필요한 경우 사용(https://wikidocs.net/40710)
        * LDA 실행 함수(https://wikidocs.net/30708)
            * input
                * Corpus:
                * Num_topics:
                * Id2word:
                * Passes:
            * output
        * LDA 시각화 함수
            * pyLDAvis.gensim_models
            * parameters
                * ldaModel: 
                * Corpus:
                * Dictionary: 
        * LDA 주요 키워드 도출 함수
        * 토픽별 추세 분석 함수
        * 토픽별 Pain-Point 분석
    * 작동 특징
        * ldaAnalysis
        * document별 토픽 category 저장
        * config.py 생성: parameter 관리
* generalAnalysis
    * 주요기능
        * 전체 분석
            * 전체 비율 
            * 전체 Trend(시계열)
                * 게시글 수
                * 작성자 수
        * 특정 column별 분석
            * column별 비율
            * column별 Trend(시계열)
                * 게시글 수
                * 작성자 수

In [4]:
import os
import pandas as pd
import numpy as np
import sys

from konlpy.tag import Kkma

sys.path.append('..')

from Code.config import *
from Code.preprocessingSentence import *

ModuleNotFoundError: No module named 'config'

In [3]:
parallelPreprocessing(config['19~23세 탈모'])

UnicodeEncodeError: 'cp949' codec can't encode character '\xa0' in position 133: illegal multibyte sequence

In [3]:
tmp = pd.DataFrame({'a' : [1, np.nan, 2, np.nan],
                    'b' : ['a', 'b', np.nan, np.nan]})

NameError: name 'pd' is not defined

In [2]:
tmp

NameError: name 'tmp' is not defined